# TWC data requests

We download data historical weather data from The Weather Company API. Specifically, historical data from the `History on Demand` package
for user defined start - end dates. Documentation available [here](https://docs.google.com/document/d/12V8zpVRh15cgP0IiuQ6n875-I04_rl2J28uE0jQcblI/edit)

# API key

In [14]:
api_key = 'will_not_work_without_api_key'

In [23]:

import pandas as pd
import numpy as np

from datetime import datetime as dt, timedelta
from pathlib import Path

In [24]:
def get_twc_history_on_demand(longitudes, latitudes, date_begin, date_end, api_key):
    print(date_begin, date_end)
    # Number of forecast hours to download for each point
    twc_date_begin = date_begin.strftime("%Y%m%d%H%M")
    twc_date_end = date_end.strftime("%Y%m%d%H%M")
    # we need to convert the date_begin/end to correct string
    pointType = 'weighted'
    geocode = '{},{}'.format(latitudes, longitudes)

    url_hod = 'https://api.weather.com/v3/wx/hod/reanalysis/historical/point?pointType={}&geocode={}' \
              '&distance=50&startDateTime={}&endDateTime={}&units=m&format=json&apiKey={}'.format(
        pointType, geocode, twc_date_begin, twc_date_end, api_key)
    print(url_hod)
    r = requests.get(url_hod)
    try:
        weather = json.loads(r.text)
        json.dumps(weather, indent=1)
        df = pd.DataFrame.from_dict(weather)
        df.set_index('observationTimeUtcIso', inplace=True)
        df = df.drop_duplicates()  # some dates are duplicated
        return df
    except ValueError:  # includes simplejson.decoder.JSONDecodeError
        print('Json decoding has failed for', twc_date_begin, twc_date_end)
        return -999, -999

In [25]:
def twc_historical_timeseries(start_date, end_date, longitude, latitude):
    date_begin = start_date # since this change inside loop
    year_string = date_begin.strftime("%Y")
    # Create WeatherData folder if doesn't exist
    Path("./WeatherData").mkdir(parents=True, exist_ok=True)
    fname_out = 'WeatherData/TWC_historical_pandas_Full_lat' + str(latitude) \
              + '_lon' + str(longitude)+ "_year" + year_string + '.csv'
    init = True # create or append to pandas data frame
    # We simply iterate over the given date period in 30 day intervals and request data
    # from API. We append to dataframe and end we write to file
    while date_begin < end_date:
        date_end = date_begin + timedelta(days=30)
        if date_end > end_date:   # check that we don't beyond user specified
            date_end = end_date
        # request data from API for given lat, lon, and particular date period
        df_return = getTWCfct_new(latitude, longitude, date_begin, date_end, api_key)
        if init:
            df_weather = df_return
            init = False
        else:
            df_weather = df_weather.append(df_return, ignore_index=False)
        date_begin = date_begin + timedelta(days=30)  # TWC allows to download historical
    df_weather.to_csv(fname_out)

In [22]:

start_date = dt(2014,1,1)
end_date = dt(2022,1,1)
longitude = str(-8.883) #  Note that MOTU expects string data
latitude = str(37.017) #
twc_historical_timeseries(start_date, end_date, longitude, latitude)
#df_hod = get_twc_history_on_demand(longitude, latitude, start_date, end_date, api_key)


2021-01-01 00:00:00 2021-01-31 00:00:00
https://api.weather.com/v3/wx/hod/reanalysis/historical/point?pointType=weighted&geocode=37.017,-8.883&distance=50&startDateTime=202101010000&endDateTime=202101310000&units=m&format=json&apiKey=8383ea3e9e8c4b9283ea3e9e8c9b9213
2021-01-31 00:00:00 2021-03-01 00:00:00
https://api.weather.com/v3/wx/hod/reanalysis/historical/point?pointType=weighted&geocode=37.017,-8.883&distance=50&startDateTime=202101310000&endDateTime=202103010000&units=m&format=json&apiKey=8383ea3e9e8c4b9283ea3e9e8c9b9213
